In [ ]:
# 📅 Business Case Simulation
# 🎯 Hedef: "Milano Smart Flex" tarifesinin finansal uygulanabilirliğini kanıtlamak.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. PARAMETRELER (Önceki analizlerimizden türetilen varsayımlar)
# -------------------------------------------------------------
# 2013 Pazar Standartları
COMPETITOR_PRICE = 15.0  # Euro
COMPETITOR_DATA_CAP = 2.0 # GB
OVERAGE_COST_PER_GB = 5.0 # GB aşım ücreti (2013'te pahalıydı!)

# Bizim Tarifemiz
SMART_FLEX_PRICE = 12.0 # Euro (Daha ucuz!)
SMART_FLEX_CAP = 2.0    # GB (Temel)
# Akıllı Özellik: İş Bölgesi (Cluster 1) ve Gece kullanımı kotadan düşmez.

# Kullanıcı Davranış Profili (Analizlerimizden)
# Varsayım: Kullanıcı trafiğinin %40'ı İş Bölgesinde, %10'u Gece gerçekleşiyor.
WORK_ZONE_RATIO = 0.40
NIGHT_RATIO = 0.10
FREE_USAGE_RATIO = WORK_ZONE_RATIO + NIGHT_RATIO # %50 Trafik Bedava!

# Operatör Maliyetleri (Tahmini)
COST_PER_GB = 0.5 # Operatöre 1GB verinin maliyeti

# 2. SİMÜLASYON MOTORU
# -------------------------------------------------------------
def simulate_profitability(num_users=10000):
    print(f"🔄 {num_users} Sanal Kullanıcı İçin Simülasyon Çalıştırılıyor...")

    # Rastgele kullanıcı profilleri oluştur (1GB ile 5GB arası kullanım)
    np.random.seed(42)
    users_usage = np.random.normal(loc=2.5, scale=1.0, size=num_users)
    users_usage = np.maximum(users_usage, 0.5) # Min 0.5 GB

    results = []

    for usage in users_usage:
        # --- RAKİP SENARYOSU ---
        comp_bill = COMPETITOR_PRICE
        if usage > COMPETITOR_DATA_CAP:
            overage = np.ceil(usage - COMPETITOR_DATA_CAP)
            comp_bill += overage * OVERAGE_COST_PER_GB

        comp_cost = usage * COST_PER_GB
        comp_margin = comp_bill - comp_cost

        # --- BİZİM SENARYO (SMART FLEX) ---
        # Akıllı özellik: Kotadan yiyen kısım sadece %50
        billable_usage = usage * (1 - FREE_USAGE_RATIO)

        smart_bill = SMART_FLEX_PRICE
        # Kullanıcı 2GB kotasını aşsa bile, faturalandırılan kısım düşük olduğu için aşım ödemez!
        # Örn: 3GB kullandı -> Faturalandırılan 1.5GB -> Aşım Yok!
        if billable_usage > SMART_FLEX_CAP:
            overage = np.ceil(billable_usage - SMART_FLEX_CAP)
            smart_bill += overage * OVERAGE_COST_PER_GB

        smart_cost = usage * COST_PER_GB # Operatör tüm veriyi taşır
        smart_margin = smart_bill - smart_cost

        results.append({
            'Total_Usage_GB': usage,
            'Competitor_Bill': comp_bill,
            'Smart_Flex_Bill': smart_bill,
            'Competitor_Margin': comp_margin,
            'Smart_Flex_Margin': smart_margin
        })

    return pd.DataFrame(results)

df_sim = simulate_profitability()

# 3. SONUÇLAR VE GÖRSELLEŞTİRME
# -------------------------------------------------------------
print("\n📊 FİNANSAL KARŞILAŞTIRMA RAPORU")
print("="*60)

avg_usage = df_sim['Total_Usage_GB'].mean()
avg_comp_bill = df_sim['Competitor_Bill'].mean()
avg_smart_bill = df_sim['Smart_Flex_Bill'].mean()
avg_comp_margin = df_sim['Competitor_Margin'].mean()
avg_smart_margin = df_sim['Smart_Flex_Margin'].mean()

print(f"Ortalama Kullanım: {avg_usage:.2f} GB/Ay")
print("-" * 30)
print(f"RAKİP TARİFE:")
print(f"  • Ort. Fatura (Kullanıcı Öder): {avg_comp_bill:.2f}€")
print(f"  • Ort. Kâr (Operatör Kazanır):  {avg_comp_margin:.2f}€")
print("-" * 30)
print(f"BİZİM TARİFE (SMART FLEX):")
print(f"  • Ort. Fatura (Kullanıcı Öder): {avg_smart_bill:.2f}€")
print(f"  • Ort. Kâr (Operatör Kazanır):  {avg_smart_margin:.2f}€")
print("="*60)

# Kullanıcı Tasarrufu
savings = avg_comp_bill - avg_smart_bill
print(f"💡 SONUÇ: Kullanıcı ayda ortalama {savings:.2f}€ tasarruf ederken,")
print(f"   Operatör sürümden kazanarak pazar payını artırır.")

# Grafik 1: Fatura Karşılaştırması
plt.figure(figsize=(10, 6))
sns.histplot(df_sim['Competitor_Bill'], color='red', alpha=0.3, label='Rakip Fatura', bins=10)
sns.histplot(df_sim['Smart_Flex_Bill'], color='green', alpha=0.5, label='Smart Flex Fatura', bins=10)
plt.title('Kullanıcı Faturası Dağılımı (Daha Düşük = Daha İyi)')
plt.xlabel('Aylık Fatura (€)')
plt.legend()
plt.show()

# Grafik 2: Kullanım vs Fatura (Scatter)
plt.figure(figsize=(10, 6))
plt.scatter(df_sim['Total_Usage_GB'], df_sim['Competitor_Bill'], alpha=0.1, color='red', label='Rakip')
plt.scatter(df_sim['Total_Usage_GB'], df_sim['Smart_Flex_Bill'], alpha=0.1, color='green', label='Smart Flex')
plt.title('Veri Kullanımı vs Fatura Tutarı')
plt.xlabel('Kullanılan Veri (GB)')
plt.ylabel('Fatura (€)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()